In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
from collections import Counter
from sklearn.model_selection import train_test_split

## Decision tree:

In [ ]:
import numpy as np
from collections import Counter

class DecisionTree:
    class _Node:
        def __init__(
            self,
            is_leaf=False,
            value=None,
            feature_index=None,
            threshold=None,
            left=None,
            right=None
        ):
            self.is_leaf = is_leaf
            self.value = value
            self.feature_index = feature_index
            self.threshold = threshold
            self.left = left
            self.right = right

    def __init__(
        self,
        classification: bool = True,
        max_depth: int = None,
        min_samples_split: int = 2,
        min_samples_leaf: int = 1,
        criterion: str = None,
        verbose: bool = False,
    ):

        self.classification = classification   # true -> classification; false -> regression
        self.max_depth = max_depth             # None -> unlimited
        self.min_samples_split = min_samples_split  # minimum sample number to continue splitting
        self.min_samples_leaf = min_samples_leaf    # minimum sample number in each leaf
        self.verbose = verbose

        if criterion is None:
            if classification:
                criterion = "gini"
            else:
                criterion = "mse"
        self.criterion = criterion

        self.root = None

    # --------- PUBLIC ---------

    def fit(self, X, y):
        """
        X: numpy array shape (n_samples, n_features)
        y: numpy array shape (n_samples,)
        """
        X = np.array(X)
        y = np.array(y)
        self.n_classes_ = None
        if self.classification:
            self.n_classes_ = len(np.unique(y))

        if self.verbose:
            print(f"[FIT] Start training")
            print(f"      n_samples={len(y)}, n_features={X.shape[1]}, "
                  f"classification={self.classification}, criterion={self.criterion}")

        # build tree (chỉ gọi 1 lần)
        self.root = self._build_tree(X, y, depth=0)

        if self.verbose:
            print("[FIT] Done building tree.")

    def predict(self, X):
        X = np.array(X)
        preds = [self._predict_one(x, self.root) for x in X]  # predict one
        return np.array(preds)

    # --------- INTERNAL ---------

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape

        if self.verbose:
            # Giảm log nếu depth quá lớn thì có thể thêm điều kiện: if depth <= 5:
            print(f"[NODE] depth={depth}, n_samples={n_samples}, "
                  f"unique_labels={np.unique(y)}")

        # Stop condition
        if self._should_stop(y, depth, n_samples):
            leaf_value = self._leaf_value(y)  # take leaf's value
            if self.verbose:
                print(f"  -> Leaf created at depth={depth}, value={leaf_value}")
            return self._Node(is_leaf=True, value=leaf_value)  # return leaf node

        # Find best split
        best_feature, best_threshold, best_gain = self._best_split(X, y)

        # Cannot find best split -> leaf
        if best_feature is None:
            leaf_value = self._leaf_value(y)
            if self.verbose:
                print(f"  -> No valid split at depth={depth}, "
                      f"create leaf with value={leaf_value}")
            return self._Node(is_leaf=True, value=leaf_value)

        # Split data
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask  # right mask = X > best_threshold

        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]

        if self.verbose:
            print(f"  -> Best split: feature={best_feature}, "
                  f"threshold={best_threshold}, gain={best_gain:.6f}")
            print(f"     Left n={len(y_left)}, Right n={len(y_right)}")

        # Buid child (create recursion)
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return self._Node(  # return internal node
            is_leaf=False,
            value=None,
            feature_index=best_feature,
            threshold=best_threshold,
            left=left_child,
            right=right_child
        )

    # stop condition
    def _should_stop(self, y, depth, n_samples):
        # 1) depth > max_depth
        if self.max_depth is not None and depth >= self.max_depth:
            if self.verbose:
                print(f"[STOP] depth={depth} reached max_depth={self.max_depth}")
            return True

        # 2) Dont have enough samples to split
        if n_samples < self.min_samples_split:
            if self.verbose:
                print(f"[STOP] n_samples={n_samples} < min_samples_split={self.min_samples_split}")
            return True

        # 3) (if classification) same labels in samples
        if self.classification and len(np.unique(y)) == 1:
            if self.verbose:
                print(f"[STOP] All samples have same label={y[0]}")
            return True

        # 4) regression (not yet)
        return False

    # --------- LEAF VALUE ---------

    def _leaf_value(self, y):
        if self.classification:
            # majority vote
            counter = Counter(y)
            return counter.most_common(1)[0][0]
        else:
            # regression: mean
            return float(np.mean(y))

    # --------- IMPURITY ---------

    def _impurity(self, y):
        if self.classification:
            if self.criterion == "gini":
                return self._gini(y)
            elif self.criterion == "entropy":
                return self._entropy(y)
            else:
                raise ValueError(f"Unknown criterion {self.criterion}")
        else:
            if self.criterion == "mse":
                return self._mse(y)
            else:
                raise ValueError(f"Unknown criterion {self.criterion}")

    def _gini(self, y):
        # Gini = 1 - sum(p_k^2)
        counts = np.bincount(y) if y.dtype == int or y.dtype == np.int64 else \
            np.array(list(Counter(y).values()))
        probs = counts / len(y)
        return 1.0 - np.sum(probs ** 2)

    def _entropy(self, y):
        # Entropy = -sum(p_k log2 p_k)
        counts = np.bincount(y) if y.dtype == int or y.dtype == np.int64 else \
            np.array(list(Counter(y).values()))
        probs = counts / len(y)
        probs = probs[probs > 0]
        return -np.sum(probs * np.log2(probs))

    def _mse(self, y):
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    # --------- FIND BEST SPLIT ---------

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        parent_impurity = self._impurity(y)
        best_gain = 0.0
        best_feature = None
        best_threshold = None

        for feature_idx in range(n_features):  # repeat through each feature
            values = X[:, feature_idx]
            thresholds = np.unique(values)
            for t in thresholds:  # try using each value that appears in the column as a threshold
                left_mask = values <= t
                right_mask = ~left_mask

                if left_mask.sum() < self.min_samples_leaf or right_mask.sum() < self.min_samples_leaf:
                    continue

                y_left, y_right = y[left_mask], y[right_mask]
                impurity_left = self._impurity(y_left)
                impurity_right = self._impurity(y_right)

                p_left = len(y_left) / len(y)
                p_right = 1.0 - p_left

                gain = parent_impurity - (p_left * impurity_left + p_right * impurity_right)  # calculate branch

                if gain > best_gain:
                    best_gain = gain  # save best branch
                    best_feature = feature_idx  # save best branch's feature
                    best_threshold = t  # save best branch's threshold

        return best_feature, best_threshold, best_gain

    def _predict_one(self, x, node):  # use decision tree to predict value (or label)
        while not node.is_leaf:
            if x[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value


### Fast test Decision tree

classification

In [ ]:
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])
y = np.array([0, 1, 1, 1])

tree_clf = DecisionTree(
    classification=True,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    criterion="gini",
)

tree_clf.fit(X, y)
print("Predict:", tree_clf.predict(X))
print("True y :", y)


Predict: [0 1 1 1]
True y : [0 1 1 1]


regression

In [ ]:
# y = 2 * x + noise
rng = np.random.RandomState(42)
X_reg = rng.rand(50, 1) * 10  # 0..10
y_reg = 2 * X_reg[:, 0] + rng.randn(50) * 0.5

tree_reg = DecisionTree(
    classification=False,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    criterion="mse",
)

tree_reg.fit(X_reg, y_reg)
preds = tree_reg.predict(X_reg)

# caculate MSE
mse = np.mean((preds - y_reg) ** 2)
print("MSE:", mse)

MSE: 0.46979596460247813


## Random Forest

In [ ]:
import numpy as np
from collections import Counter
# import DecisionTree

class RandomForest:
    def __init__(
        self,
        classification: bool = True,
        n_trees: int = 10,
        max_depth: int = None,
        min_samples_split: int = 2,
        min_samples_leaf: int = 1,
        max_features: int = None,
        bootstrap: bool = True,
        random_state: int = None,
    ):
        self.classification = classification
        self.n_trees = n_trees # number of tree
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split  # min samples to split
        self.min_samples_leaf = min_samples_leaf    # min sapmles in leaf
        self.max_features = max_features            # max features to consider when looking for the best split
        self.bootstrap = bootstrap                  # Whether to use bootstrap samples for each tree.
        self.random_state = random_state

        self.trees = []
        self.n_features_ = None
        self.feature_indices_per_tree = []

        if random_state is not None:
            np.random.seed(random_state)

    def fit(self, X, y):
        """
        Fit the Random Forest on data X, y.
        """
        X = np.array(X)
        y = np.array(y)
        n_samples, n_features = X.shape
        self.n_features_ = n_features

        # If max_features is None, use all features
        if self.max_features is None or self.max_features > n_features:
            self.max_features = n_features

        self.trees = []
        self.feature_indices_per_tree = []

        for i in range(self.n_trees):
            # Select feature subset (feature randomness)
            feature_indices = np.random.choice(
                n_features,
                size=self.max_features,
                replace=False
            )
            self.feature_indices_per_tree.append(feature_indices)

            # Bootstrap sampling for rows
            if self.bootstrap:
                sample_indices = np.random.choice(
                    n_samples,
                    size=n_samples,
                    replace=True
                )
            else:
                sample_indices = np.arange(n_samples)

            X_boot = X[sample_indices][:, feature_indices]
            y_boot = y[sample_indices]

            # Create a new DecisionTree and fit it
            tree = DecisionTree(
                classification=self.classification,
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                criterion="gini" if self.classification else "mse",
            )
            tree.fit(X_boot, y_boot) # fit
            self.trees.append(tree)

    def predict(self, X):
        """
        Predict for data X using all trees and aggregate the predictions.
        """
        X = np.array(X)
        n_samples = X.shape[0]

        # Collect predictions from each tree
        all_preds = []
        for tree, feat_idx in zip(self.trees, self.feature_indices_per_tree):
            X_sub = X[:, feat_idx]
            preds = tree.predict(X_sub)
            all_preds.append(preds)

        all_preds = np.array(all_preds)  # shape: (n_trees, n_samples)

        if self.classification:
            # Majority vote along axis 0
            final_preds = []
            for i in range(n_samples):
                votes = all_preds[:, i]
                counter = Counter(votes)
                final_preds.append(counter.most_common(1)[0][0])
            return np.array(final_preds)
        else:
            # Regression: average over trees
            return np.mean(all_preds, axis=0)


### Fast test Random Forest

In [ ]:
# Classification test
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])
y = np.array([0, 1, 1, 1])

rf_clf = RandomForest(
    classification=True,
    n_trees=5,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=2,
    bootstrap=True,
    random_state=42,
)

rf_clf.fit(X, y)
preds = rf_clf.predict(X)

print("X:")
print(X)
print("True labels :", y)
print("RF predictions:", preds)
print("Accuracy      :", np.mean(preds == y))


X:
[[0 0]
 [0 1]
 [1 0]
 [1 1]]
True labels : [0 1 1 1]
RF predictions: [0 1 1 1]
Accuracy      : 1.0


In [ ]:
# Toy regression: y = 2 * x + noise
rng = np.random.RandomState(42)
X_reg = rng.rand(100, 1) * 10  # range 0..10
y_reg = 2 * X_reg[:, 0] + rng.randn(100) * 0.5

rf_reg = RandomForest(
    classification=False,
    n_trees=10,
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=1,
    bootstrap=True,
    random_state=42,
)

rf_reg.fit(X_reg, y_reg)
preds_reg = rf_reg.predict(X_reg)

mse_rf = np.mean((preds_reg - y_reg) ** 2)
print("RandomForest Regression MSE:", mse_rf)


RandomForest Regression MSE: 0.1050399719860864


## Gradient Boosting

In [ ]:
import numpy as np
# import DecisionTree

class GradientBoosting:
    def __init__(
        self,
        classification: bool = True,
        n_estimators: int = 50,
        learning_rate: float = 0.1,
        max_depth: int = 3,
        min_samples_split: int = 2,
        min_samples_leaf: int = 1,
        random_state: int = None,
    ):
        self.classification = classification
        self.n_estimators = n_estimators    #  Number of boosting stages (trees).
        self.learning_rate = learning_rate  # each tree contributes 'lr' its predict's turn number
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

        if random_state is not None:
            np.random.seed(random_state)

        self.trees = []          # list of trained tree
        self.init_value_ = None  # initial prediction (bias term)

    def fit(self, X, y):
        """
        Fit Gradient Boosting model on data X, y.
        Uses MSE loss for both regression and binary classification (0/1).
        """
        X = np.array(X)
        y = np.array(y, dtype=float)  # cast to float for regression-style updates

        n_samples = X.shape[0]

        # Initialize prediction:
        # For regression: mean of y
        # For binary classification: mean of y in [0, 1] range (approximate prob)
        self.init_value_ = np.mean(y)
        current_pred = np.full(n_samples, self.init_value_, dtype=float)

        self.trees = []

        for m in range(self.n_estimators):  # for each tree
            # Residuals for MSE: r = y - y_pred
            residuals = y - current_pred

            # Train a small regression tree on residuals
            tree = DecisionTree(
                classification=False,  # always regression tree for residuals
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                criterion="mse",
            )
            tree.fit(X, residuals)

            # Get tree predictions
            tree_pred = tree.predict(X)

            # Update current predictions
            current_pred += self.learning_rate * tree_pred

            # Store tree
            self.trees.append(tree)

    def _raw_predict(self, X):
        """
        Compute raw prediction (regression output) before any thresholding.
        """
        X = np.array(X)
        # Start with initial value
        pred = np.full(X.shape[0], self.init_value_, dtype=float)
        # Add contributions from all trees
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        return pred

    def predict(self, X):
        """
        Predict for data X.

        For regression: return raw predictions.
        For classification: return class labels 0 or 1 (binary).
        """
        raw_pred = self._raw_predict(X)

        if self.classification:
            # Binary classification: threshold at 0.5
            return (raw_pred >= 0.5).astype(int)
        else:
            # Regression: return continuous value
            return raw_pred


### Fast test GB

In [ ]:
# Simple binary classification toy data
rng = np.random.RandomState(0)
n_samples = 200

# Class 0 around (0,0)
X0 = rng.randn(n_samples // 2, 2) + np.array([-1, -1])
y0 = np.zeros(n_samples // 2, dtype=int)

# Class 1 around (2,2)
X1 = rng.randn(n_samples // 2, 2) + np.array([2, 2])
y1 = np.ones(n_samples // 2, dtype=int)

X_clf = np.vstack([X0, X1])
y_clf = np.concatenate([y0, y1])

gb_clf = GradientBoosting(
    classification=True,
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=0,
)

gb_clf.fit(X_clf, y_clf)
preds_clf = gb_clf.predict(X_clf)

acc = np.mean(preds_clf == y_clf)
print("GradientBoosting Classification Accuracy:", acc)


GradientBoosting Classification Accuracy: 1.0


In [ ]:
# Toy regression: y = 2 * x + noise
rng = np.random.RandomState(42)
X_reg = rng.rand(200, 1) * 10  # range 0..10
y_reg = 2 * X_reg[:, 0] + rng.randn(200) * 0.5

gb_reg = GradientBoosting(
    classification=False,
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
)

gb_reg.fit(X_reg, y_reg)
preds_reg = gb_reg.predict(X_reg)

mse_gb = np.mean((preds_reg - y_reg) ** 2)
print("GradientBoosting Regression MSE:", mse_gb)


GradientBoosting Regression MSE: 0.11912061804868895


## Work with dataset London-Bike

### Preparing steps

Load dataset from file CSV (London Bike)

In [ ]:
csv_path = "/content/london_merged.csv"  # adjust if needed

df = pd.read_csv(csv_path)
df.head()

# timestamp - time
# cnt - count of a new bike shares
# t1 - real temperature in C
# t2 - temperature in C "feels like"
# hum - humidity in percentage
# wind_speed - wind speed in km/h
# weather_code - category of the weather
# is_holiday - boolean field - 1 holiday / 0 non holiday
# is_weekend -  boolean field - 1 if the day is weekend
# season - 0-spring ; 1-summer; 2-fall; 3-winter

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


choose features & target; preprocessing data
* target: cnt

In [ ]:
# Choose target column (bike count)
target_col = "cnt"

# Choose a set of numeric and categorical features
feature_cols = [
    "t1",           # real temperature
    "t2",           # feels-like temperature
    "hum",          # humidity
    "wind_speed",   # wind speed
    "weather_code", # weather condition code
    "is_holiday",
    "is_weekend",
    "season",
]

# Filter dataframe (drop rows with missing values just in case)
df_model = df[feature_cols + [target_col]].dropna()

X = df_model[feature_cols].values
y = df_model[target_col].values

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (17414, 8)
y shape: (17414,)


split train & test samples

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
)

print("Train size:", X_train.shape[0])
print("Test size :", X_test.shape[0])


Train size: 13931
Test size : 3483


### Utils

In [ ]:
def regression_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    mse = np.mean((y_true - y_pred) ** 2)
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(mse)

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0

    return {
        "MAE": float(mae),
        "MSE": float(mse),
        "RMSE": float(rmse),
        "R2": float(r2),
    }


### Train with DecisionTree

In [ ]:
# Decision Tree Regressor
dt_reg = DecisionTree(
    classification=False,
    max_depth=6,           # can tune this
    min_samples_split=10,  # to avoid overfitting
    min_samples_leaf=5,
    criterion="mse",
)

dt_reg.fit(X_train, y_train)

# Predictions on train and test
y_train_pred_dt = dt_reg.predict(X_train)
y_test_pred_dt = dt_reg.predict(X_test)


### Train with RandomForest

In [ ]:
# Random Forest Regressor using our custom RandomForest
rf_reg = RandomForest(
    classification=False,
    n_trees=10,           # can be increased if it is not too slow
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features=None,    # use all features here
    bootstrap=True,
    random_state=42,
)

rf_reg.fit(X_train, y_train)

y_train_pred_rf = rf_reg.predict(X_train)
y_test_pred_rf = rf_reg.predict(X_test)


### Train with GradientBoosting

In [ ]:
gb_reg = GradientBoosting(
    classification=False,
    n_estimators=50,   # number of boosting stages
    learning_rate=0.1,
    max_depth=3,       # shallow trees as "stumps"
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
)

gb_reg.fit(X_train, y_train)

y_train_pred_gb = gb_reg.predict(X_train)
y_test_pred_gb = gb_reg.predict(X_test)


### Metric comparison  

In [ ]:
print("=== London Bike Regression: Comparison ===")

print("=== DecisionTree Regression metrics ===")
dt_train_metrics = regression_metrics(y_train, y_train_pred_dt)
dt_test_metrics  = regression_metrics(y_test,  y_test_pred_dt)
# print("Train:", dt_train_metrics)
print("Test :", dt_test_metrics)
print()

print("=== RandomForest Regression metrics ===")
rf_train_metrics = regression_metrics(y_train, y_train_pred_rf)
rf_test_metrics  = regression_metrics(y_test,  y_test_pred_rf)
# print("Train:", rf_train_metrics)
print("Test :", rf_test_metrics)
print()

print("=== GradientBoosting Regression metrics ===")
gb_train_metrics = regression_metrics(y_train, y_train_pred_gb)
gb_test_metrics  = regression_metrics(y_test,  y_test_pred_gb)
# print("Train:", gb_train_metrics)
print("Test :", gb_test_metrics)



=== London Bike Regression: Comparison ===
=== DecisionTree Regression metrics ===
Test : {'MAE': 667.2574515022576, 'MSE': 848428.6819700071, 'RMSE': 921.1018846848633, 'R2': 0.29193936375589336}

=== RandomForest Regression metrics ===
Test : {'MAE': 658.6768159233443, 'MSE': 826972.0131750309, 'RMSE': 909.3800158212357, 'R2': 0.30984613999001753}

=== GradientBoosting Regression metrics ===
Test : {'MAE': 658.3612702538719, 'MSE': 825637.2993238627, 'RMSE': 908.6458602359131, 'R2': 0.3109600324818034}


## **Анализ результатов на датасете London-Bike**
### **1. DecisionTree**

* Одиночное дерево имеет **высокую дисперсию**.
* Оно слишком точно подстраивается под шум обучающей выборки.
* Способность к обобщению низкая.
* Это — хорошо известный недостаток классических решающих деревьев.


### **2. RandomForest**

* RandomForest **снижает дисперсию** модели.
* Выполняет усреднение предсказаний большого числа независимых деревьев.
* Bootstrap-выборки помогают избежать переобучения.
* Предсказания получаются более стабильными.


### **3. GradientBoosting**

* Boosting учит деревья **на остатках (residuals)**, постепенно улучшая модель.
* Это последовательная схема обучения, исправляющая ошибки предыдущих деревьев.
* Подходит для **нелинейных регрессионных задач**.
* Параметры *learning rate = 0.1* и *max_depth = 3* дают хороший баланс качества и устойчивости.
* Переобучения удаётся избежать, если число этапов (деревьев) не слишком велико.


## **Вывод**: DecisionTree < RandomForest < GradientBoosting


## Work with dataset CIFAR

### Preparing steps

#### Load dataset

In [ ]:
!tar -xzf cifar-10-python.tar.gz -C /content/
!ls

cifar-10-batches-py  cifar-10-python.tar.gz  sample_data


function load 1 batch CIFAR-10

In [ ]:
def load_cifar_batch(file_path):
    """
    Load a single CIFAR-10 batch file.
    """
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='latin1')  # encoding for Python3

    data = batch['data']            # shape (10000, 3072)
    labels = batch['labels']        # list of int labels (0..9)
    data = np.array(data, dtype=np.uint8)
    labels = np.array(labels, dtype=np.int64)
    return data, labels


Load all train + test CIFAR-10

In [ ]:
cifar_dir = "/content/cifar-10-batches-py"

# Load train batches (data_batch_1..5)
X_list = []
y_list = []

for i in range(1, 6):
    batch_path = os.path.join(cifar_dir, f"data_batch_{i}")
    X_batch, y_batch = load_cifar_batch(batch_path)
    X_list.append(X_batch)
    y_list.append(y_batch)

X_train_full = np.vstack(X_list)           # shape (50000, 3072)
y_train_full = np.concatenate(y_list)      # shape (50000,)

print("Train full shape:", X_train_full.shape, y_train_full.shape)

# Load test batch
test_path = os.path.join(cifar_dir, "test_batch")
X_test_full, y_test_full = load_cifar_batch(test_path)

print("Test full shape:", X_test_full.shape, y_test_full.shape)


Train full shape: (50000, 3072) (50000,)
Test full shape: (10000, 3072) (10000,)


#### Preprocessing data

In [ ]:
# Convert to float32 and normalize to [0, 1]
X_train_full = X_train_full.astype(np.float32) / 255.0
X_test_full  = X_test_full.astype(np.float32) / 255.0

print("After normalization:")
print("Train full:", X_train_full.shape, X_train_full.dtype)
print("Test full :", X_test_full.shape, X_test_full.dtype)

After normalization:
Train full: (50000, 3072) float32
Test full : (10000, 3072) float32


### Utils

In [ ]:
def accuracy_score(y_true, y_pred):
    """
    Simple accuracy: percentage of correct predictions.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(y_true == y_pred)

Choose a smaller, balanced CIFAR-10 subset between classes, randomly controlled,

In [ ]:
def sample_balanced(X, y, n_per_class, random_state=42):
    """
    Sample n_per_class examples for each class (0..9) from X, y.
    """
    rng = np.random.RandomState(random_state)
    classes = np.unique(y)
    X_out = []
    y_out = []

    for c in classes:
        idx = np.where(y == c)[0]
        if len(idx) < n_per_class:
            raise ValueError(f"Not enough samples for class {c}")
        chosen = rng.choice(idx, size=n_per_class, replace=False)
        X_out.append(X[chosen])
        y_out.append(y[chosen])

    X_out = np.vstack(X_out)
    y_out = np.concatenate(y_out)
    return X_out, y_out

# Choose how many per class
n_train_per_class = 300   # => * 10 train
n_test_per_class  = 100    # => * 10 test

X_train_small, y_train_small = sample_balanced(X_train_full, y_train_full, n_train_per_class, random_state=0)
X_test_small,  y_test_small  = sample_balanced(X_test_full,  y_test_full,  n_test_per_class,  random_state=1)

print("Small train:", X_train_small.shape, y_train_small.shape)
print("Small test :", X_test_small.shape,  y_test_small.shape)


Small train: (2000, 3072) (2000,)
Small test : (1000, 3072) (1000,)


### Train with DecisionTree

In [ ]:
dt_cifar = DecisionTree(
    classification=True,
    max_depth=12,        # a little more
    min_samples_split=30,
    min_samples_leaf=8,
    criterion="gini",
)

print("Training DecisionTree on CIFAR-10 subset...")
dt_cifar.fit(X_train_small, y_train_small)

y_train_pred_dt = dt_cifar.predict(X_train_small)
y_test_pred_dt  = dt_cifar.predict(X_test_small)

acc_train_dt = accuracy_score(y_train_small, y_train_pred_dt)
acc_test_dt  = accuracy_score(y_test_small,  y_test_pred_dt)

print("DecisionTree CIFAR-10")
print("Train accuracy:", acc_train_dt)
print("Test  accuracy:", acc_test_dt)

Training DecisionTree on CIFAR-10 subset...
DecisionTree CIFAR-10
Train accuracy: 0.566
Test  accuracy: 0.218


In [ ]:
# Decision Tree for CIFAR-10
dt_cifar = DecisionTree(
    classification=True,
    max_depth=10,
    min_samples_split=50,
    min_samples_leaf=20,
    criterion="gini",
)

print("Training DecisionTree on CIFAR-10 subset...")
dt_cifar.fit(X_train_small, y_train_small)

y_train_pred_dt = dt_cifar.predict(X_train_small)
y_test_pred_dt  = dt_cifar.predict(X_test_small)

acc_train_dt = accuracy_score(y_train_small, y_train_pred_dt)
acc_test_dt  = accuracy_score(y_test_small,  y_test_pred_dt)

print("DecisionTree CIFAR-10")
print("Train accuracy:", acc_train_dt)
print("Test  accuracy:", acc_test_dt)


Training DecisionTree on CIFAR-10 subset...
DecisionTree CIFAR-10
Train accuracy: 0.472
Test  accuracy: 0.225


### Train with RandomForest

In [ ]:
# Random Forest for CIFAR-10
rf_cifar = RandomForest(
    classification=True,
    n_trees=10,
    max_depth=12,
    min_samples_split=40,
    min_samples_leaf=10,
    max_features=64,
    bootstrap=True,
    random_state=42,
)

print("Training RandomForest on CIFAR-10 subset...")
rf_cifar.fit(X_train_small, y_train_small)

y_train_pred_rf = rf_cifar.predict(X_train_small)
y_test_pred_rf  = rf_cifar.predict(X_test_small)

acc_train_rf = accuracy_score(y_train_small, y_train_pred_rf)
acc_test_rf  = accuracy_score(y_test_small,  y_test_pred_rf)

print("RandomForest CIFAR-10")
print("Train accuracy:", acc_train_rf)
print("Test  accuracy:", acc_test_rf)


Training RandomForest on CIFAR-10 subset...
RandomForest CIFAR-10
Train accuracy: 0.6145
Test  accuracy: 0.269


### Train with Gradient Boostring

 **Method “One-vs-Rest”**:

Separate the multi-layered problem into multiple binary problems.
For each class c, see:

* class c → label 1

* other class → label 0

In [ ]:
class OneVsRestGB:
    def __init__(self, n_classes, n_estimators=20, learning_rate=0.1,
                 max_depth=3, min_samples_split=10, min_samples_leaf=5,
                 random_state=42):
        """
        One-vs-Rest wrapper for GradientBoosting on multi-class tasks.
        Trains one regression boosting model per class.
        """
        self.n_classes = n_classes
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state
        self.models = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.models = []

        for c in range(self.n_classes):
            # Binary labels: 1 for class c, 0 for others
            y_bin = (y == c).astype(float)
            gb = GradientBoosting(
                classification=False,          # use regression mode on 0/1
                n_estimators=self.n_estimators,
                learning_rate=self.learning_rate,
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                random_state=self.random_state,
            )
            gb.fit(X, y_bin)
            self.models.append(gb)

    def predict(self, X):
        X = np.array(X)
        # For each class model, get raw prediction (probability-like score)
        scores = []
        for gb in self.models:
            s = gb._raw_predict(X)          # shape (n_samples,)
            scores.append(s)
        scores = np.vstack(scores).T        # shape (n_samples, n_classes)
        # Choose class with highest score
        return np.argmax(scores, axis=1)


Train

In [ ]:
n_classes = len(np.unique(y_train_small))

gb_cifar = OneVsRestGB(
    n_classes=10,
    n_estimators=8,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=50,
    min_samples_leaf=20,
    random_state=42,
)

print("Training GradientBoosting (One-vs-Rest) on CIFAR-10 subset...")
gb_cifar.fit(X_train_small, y_train_small)

y_train_pred_gb = gb_cifar.predict(X_train_small)
y_test_pred_gb  = gb_cifar.predict(X_test_small)

acc_train_gb = accuracy_score(y_train_small, y_train_pred_gb)
acc_test_gb  = accuracy_score(y_test_small,  y_test_pred_gb)

print("GradientBoosting (OvR) CIFAR-10")
print("Train accuracy:", acc_train_gb)
print("Test  accuracy:", acc_test_gb)


Training GradientBoosting (One-vs-Rest) on CIFAR-10 subset...
GradientBoosting (OvR) CIFAR-10
Train accuracy: 0.623
Test  accuracy: 0.308


### Metric comparison

In [ ]:
print("=== CIFAR-10 Accuracy Comparison (subset) ===")
print(f"DecisionTree   - Train: {acc_train_dt:.4f} | Test: {acc_test_dt:.4f}")
print(f"RandomForest   - Train: {acc_train_rf:.4f} | Test: {acc_test_rf:.4f}")
print(f"GradientBoost. - Train: {acc_train_gb:.4f} | Test: {acc_test_gb:.4f}")

=== CIFAR-10 Accuracy Comparison (subset) ===
DecisionTree   - Train: 0.4720 | Test: 0.2250
RandomForest   - Train: 0.6145 | Test: 0.2690
GradientBoost. - Train: 0.6230 | Test: 0.3080




## **Анализ результатов на датасете CIFAR-10**


### **1. DecisionTree**

* *Train accuracy = 0.472*, *Test accuracy = 0.225*.
* Одиночное дерево сильно ограничено на задачах с высокоразмерными данными (3072 признака).
* Модель имеет высокую дисперсию и переобучается, даже несмотря на ограничения глубины.
* Способность к обобщению низкая, поэтому тестовая точность остаётся около 22–23%.

### **2. RandomForest**

* *Train accuracy = 0.6145*, *Test accuracy = 0.269*.
* За счёт bagging и случайного подбора признаков RandomForest существенно уменьшает дисперсию по сравнению с одиночным деревом.
* Модель становится более устойчивой и менее чувствительной к шуму.
* Улучшение тестовой точности до ~0.27 подтверждает, что ансамбль деревьев обобщает данные лучше, чем одно дерево.

### **3. GradientBoosting**

* *Train accuracy = 0.623*, *Test accuracy = 0.308*.
* Boosting обучается последовательно, исправляя ошибки предыдущих деревьев, что позволяет моделировать более сложные нелинейные зависимости.
* Несмотря на небольшую глубину деревьев, boosting эффективно адаптируется под структуру данных.
* Лучшая тестовая точность (~0.31) показывает, что модель наиболее хорошо захватывает информативные признаки даже при ограниченной выборке.



## **Вывод**: DecisionTree < RandomForest < GradientBoosting
